In [1]:
%pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [2]:
from transformers import BertTokenizer, BertForSequenceClassification

2025-04-15 22:46:09.419546: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744757169.726885   16073 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744757169.829269   16073 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-15 22:46:10.502720: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
#Carregue o tokenizer e o modelo:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
#Prepare um texto para análise:

text = "I love using BERT for natural language processing!"
inputs = tokenizer(text, return_tensors='pt')

In [7]:
#Faça a previsão:

outputs = model(**inputs)
logits = outputs.logits
predicted_class = logits.argmax().item()

In [8]:
#Para análise de sentimentos, o BERT pode ser adaptado para classificar textos em categorias como positivo, negativo ou neutro:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Carregue o modelo e o tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)  # 3 classes: positivo, negativo, neutro# Texto para análise
text = "Este produto é incrível, superou todas as minhas expectativas!"

# Tokenize o texto
inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)

# Faça a previsão
with torch.no_grad():
    outputs = model(**inputs)
    predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
    
# Obtenha a classe com maior probabilidade
predicted_class = predictions.argmax().item()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
#Implementando BERT para Inferência de Tokens Mascarados

from transformers import BertTokenizer, BertForMaskedLM
import torch

# Carregue o modelo e o tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

# Texto com uma palavra mascarada
text = "O [MASK] é a capital do Brasil."
inputs = tokenizer(text, return_tensors='pt')

# Obtenha o índice do token mascarado
mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]

# Faça a previsão
with torch.no_grad():
    outputs = model(**inputs)
    predictions = outputs.logits

# Obtenha as 5 palavras mais prováveis para o token mascarado
predicted_token_ids = torch.topk(predictions[0, mask_token_index], 5).indices
for token_id in predicted_token_ids:
    print(tokenizer.decode([token_id]))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


TypeError: int() argument must be a string, a bytes-like object or a real number, not 'list'